<a href="https://colab.research.google.com/github/yasirsid2004/Whats-for-Dinner-/blob/main/Whats_for_Dinner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Cell 1: Install the required libraries
!pip install python-telegram-bot nest_asyncio requests

In [3]:
# Cell 2: Import libraries and prepare the environment

import logging
import nest_asyncio
import requests
from google.colab import userdata

# --- Imports for Conversation Bot ---
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (
    ApplicationBuilder,
    ContextTypes,
    CommandHandler,
    MessageHandler,
    filters,
    CallbackQueryHandler,  # To handle button clicks
    ConversationHandler    # To manage the multi-step conversation
)
# ---------------------

# This line allows the bot's event loop to run correctly inside Colab
nest_asyncio.apply()

# --- Load BOTH of your secret keys ---
TELEGRAM_TOKEN = userdata.get('TELEGRAM_TOKEN')
SPOONACULAR_KEY = userdata.get('SPOONACULAR_KEY')
# -------------------------------------

# Set up basic logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# --- Define the "states" for our conversation ---
# We still only have ONE state
SELECTING_INGREDIENTS = 1
# ------------------------------------------------

print("✅ Libraries v9.0 (Spoonacular) imported and keys are loaded.")

✅ Libraries v9.0 (Spoonacular) imported and keys are loaded.


In [4]:
# Cell 3: Define the bot's logic (v9.1 - With Signature)

# --- Define our single, large list of ingredients ---
ALL_INGREDIENTS = [
    # Meats
    "chicken", "beef", "pork", "lamb", "salmon",
    # Staples
    "egg", "rice", "pasta", "potato",
    # Veggies
    "onion", "garlic", "tomato", "carrot", "bell pepper",
    # Dairy & Other
    "cheese", "milk", "butter", "olive oil"
]

# --- Helper Function to build the ingredient keyboard ---
def build_ingredient_keyboard(selected_ingredients):
    """Creates the keyboard with a '✅' next to selected ingredients."""
    keyboard = []
    row = []

    for ingredient in ALL_INGREDIENTS:
        text = f"{'✅ ' if ingredient in selected_ingredients else ''}{ingredient.capitalize()}"
        row.append(InlineKeyboardButton(text, callback_data=f"ing_{ingredient}"))

        if len(row) == 3:
            keyboard.append(row)
            row = []
    if row:
        keyboard.append(row)

    keyboard.append([InlineKeyboardButton("➡️ DONE - Search Recipes ⬅️", callback_data="done")])
    return InlineKeyboardMarkup(keyboard)

# --- Conversation Step 1: Start ---
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Starts the conversation and shows the main ingredient checklist."""
    context.user_data["selected_ingredients"] = set()
    keyboard = build_ingredient_keyboard(context.user_data["selected_ingredients"])

    await update.message.reply_text(
        "👋 Hi! I'm your Spoonacular-powered recipe bot (v9.1).\n\n"
        "Please click **all** the ingredients you have. "
        "Click 'Done' when you're finished.",
        reply_markup=keyboard
    )
    return SELECTING_INGREDIENTS

# --- Conversation Step 2: Handle Ingredient Clicks ---
async def ingredient_choice(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handles clicks on the ingredient grid, adding/removing them from the list."""
    query = update.callback_query
    await query.answer()

    ingredient = query.data.split('_', 1)[1]

    if ingredient in context.user_data["selected_ingredients"]:
        context.user_data["selected_ingredients"].remove(ingredient)
    else:
        context.user_data["selected_ingredients"].add(ingredient)

    keyboard = build_ingredient_keyboard(context.user_data["selected_ingredients"])
    await query.edit_message_text(
        text="Please click **all** the ingredients you have. "
             "Click 'Done' when you're finished.",
        reply_markup=keyboard,
    )
    return SELECTING_INGREDIENTS

# --- Conversation Step 3: Handle "Done" button and Search (NEW) ---
async def search_recipes(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """User clicked 'Done'. Search Spoonacular with all ingredients."""
    query = update.callback_query
    await query.answer()

    selected_ingredients = context.user_data.get("selected_ingredients")

    if not selected_ingredients:
        await query.edit_message_text("You didn't select any ingredients! Try again by sending /start.")
        context.user_data.clear()
        return ConversationHandler.END

    await query.edit_message_text(
        f"OK! Connecting to Spoonacular... Searching for recipes with:\n"
        f"*{', '.join(selected_ingredients)}*...",
        parse_mode='Markdown'
    )

    # --- SPOONACULAR API CALL ---
    ingredients_string = ",".join(selected_ingredients)
    API_URL = "https://api.spoonacular.com/recipes/findByIngredients"
    params = {
        "apiKey": SPOONACULAR_KEY,
        "ingredients": ingredients_string,
        "number": 10,
        "ranking": 2
    }

    try:
        response = requests.get(API_URL, params=params)
        recipes = response.json()

        if recipes and len(recipes) > 0:
            reply_text = f"✅ Success! I found **{len(recipes)}** recipes for you:\n\n"
            for recipe in recipes:
                recipe_name = recipe['title']
                missing_count = recipe['missedIngredientCount']

                if missing_count == 0:
                    reply_text += f"• **{recipe_name}** (You have all the ingredients!)\n"
                else:
                    reply_text += f"• **{recipe_name}** (You're only missing {missing_count} ingredients)\n"

            # --- THIS IS YOUR NEW SIGNATURE ---
            reply_text += "\n\n*- Made by Yasir Siddiqui -*"
            # ------------------------------------

        else:
            reply_text = "😢 Sorry, even the Spoonacular database couldn't find a match for that combination."

    except Exception as e:
        print(f"Error calling Spoonacular API: {e}")
        reply_text = "Sorry, I had an error connecting to the recipe database."
    # ------------------------------------------------

    # --- Send the final reply ---
    await context.bot.send_message(
        chat_id=update.effective_chat.id,
        text=reply_text,
        parse_mode='Markdown'
    )

    context.user_data.clear()
    return ConversationHandler.END

# --- Fallback: Cancel command ---
async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Cancels and ends the conversation."""
    await update.message.reply_text("OK, I've cancelled our current search. Send /start to begin again.")
    context.user_data.clear()
    return ConversationHandler.END

print("✅ Bot logic v9.1 (With Signature) is now defined.")

✅ Bot logic v9.1 (With Signature) is now defined.


In [ ]:
# Cell 4 (Corrected): Start the bot (Spoonacular Version)

# Create the bot application
application = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

# --- Create the ConversationHandler ---
conv_handler = ConversationHandler(
    entry_points=[CommandHandler("start", start)],
    states={
        SELECTING_INGREDIENTS: [
            CallbackQueryHandler(search_recipes, pattern="^done$"),
            CallbackQueryHandler(ingredient_choice, pattern="^ing_")
        ],
    },
    fallbacks=[CommandHandler("cancel", cancel)],
    allow_reentry=True
)
# --------------------------------------

# Add the ConversationHandler to the bot
application.add_handler(conv_handler)

# --- The non-blocking way to run the bot ---
import asyncio

async def main():
    """Initializes and runs the bot until you stop the cell."""
    await application.initialize()
    await application.updater.start_polling()
    await application.start()
    print("🚀 Bot is running (v9.0 Spoonacular!)... Open Telegram and chat!")

    await asyncio.Event().wait()

if __name__ == "__main__":
    asyncio.run(main())

/tmp/ipython-input-4035684820.py:7: PTBUserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message. Read this FAQ entry to learn more about the per_* settings: https://github.com/python-telegram-bot/python-telegram-bot/wiki/Frequently-Asked-Questions#what-do-the-per_-settings-in-conversationhandler-do.
  conv_handler = ConversationHandler(


🚀 Bot is running (v9.0 Spoonacular!)... Open Telegram and chat!


ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_async/connection.py", line 103, in handle_async_request
    return await self._connection.handle_async_request(reques